In [ ]:
%pip install rapidfuzz fancyimpute
#I used this to install two libraries. rapidfuzz helped me group similar product names with spelling
# variations — like ‘Maggi’ vs ‘Maggie’. fancyimpute offers missing value tools, but I later used my
# own imputation logic based on group means and regression

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 12.8 MB/s eta 0:00:00
  Created wheel for fancyimpute: filename=fancyimpute-0.7.0-py3-none-any.whl size=29879 sha256=9abdbcb5aa0789b759aecdcc54bb6dd3bcf9484015e738c8c5662a8273abf182
  Stored in directory: /root/.cache/pip/wheels/1a/f3/a1/f7f10b5ae2c2459398762a3fcf4ac18c325311c7e3163d5a15
  Created wheel for knnimpute: filename=knnimpute-0.1.0-py3-none-any.whl size=11331 sha256=d4c9036d3295397ccc2f788b15eb9cd9ef2344c6c14f12c228f9cc698eb4de26
  Stored in directory: /root/.cache/pip/wheels/ea/e8/e0/79872972161e54486517ae507f94b2c7cea27fb7ef793bd415
Successfully built fancyimpute knnimpute


In [ ]:
import pandas as pd, numpy as np, re
from rapidfuzz.process import cdist
from rapidfuzz import fuzz, process
from sklearn.linear_model import LinearRegression
#I imported pandas and numpy for data handling, regex for extracting sizes,
#rapidfuzz for fuzzy matching similar product names, and LinearRegression for predicting missing MRP values

In [ ]:
df = pd.read_csv('/content/FMCG_Sales_Data.csv')
df['original_name'] = df['Product Name']

In [ ]:
df['clean_name'] = (
    df['Product Name'].str.lower()
                      .str.replace(r'[^a-z0-9 ]', ' ', regex=True)
                      .str.replace(r'\s+', ' ', regex=True)
                      .str.strip()
)
#“I cleaned the product names by lowercasing them, removing symbols, fixing extra spaces, and trimming
# all to make name comparisons consistent for grouping


In [ ]:
# Getting unique names
unique_names = df['clean_name'].dropna().unique().tolist()

# Compute=ing fuzzy similarity matrix
score_cutoff = 85
scores = cdist(unique_names, unique_names, scorer=fuzz.token_set_ratio, score_cutoff=score_cutoff, workers=-1)

# Grouped the  n names based on fuzzy similarityy
clusters = []
visited = set()
for i, name in enumerate(unique_names):
    if i in visited:
        continue
    similar_indices = set(np.where(scores[i] >= score_cutoff)[0].tolist() + [i])
    visited |= similar_indices
    group = sorted([unique_names[j] for j in similar_indices])
    clusters.append(group)

#I created clusters of similar-looking product names using fuzzy string matching
# this helped combine variations like different spellings or spacing into one consistent group

In [ ]:
cluster_df = pd.DataFrame({
    'members': ['; '.join(c) for c in clusters],
    'canonical': [c[0] for c in clusters]  # Manually i will edit them
})

cluster_df.to_csv('/content/brand_clusters_filled.csv', index=False)
print("✅ Brand clusters saved. Download 'brand_clusters.csv', assign canonical names, and upload as 'brand_clusters_filled.csv'.")
#I saved all the fuzzy-matched name clusters into a CSV so I could manually assign a
#clean name for each group. This ensureed consistent brand naming throughout the analysis.

✅ Brand clusters saved. Download 'brand_clusters.csv', assign canonical names, and upload as 'brand_clusters_filled.csv'.


In [ ]:
clusters_filled = pd.read_csv('/content/brand_clusters_filled.csv')

# Explode members column into one row per alias
clusters_filled = clusters_filled.assign(member=clusters_filled['members'].str.split(';'))
clusters_filled = clusters_filled.explode('member')

# Strip whitespace from member values
clusters_filled['member'] = clusters_filled['member'].str.strip()

# Creating a dictionary mapping from member → canonical
brand_map = dict(zip(clusters_filled['member'], clusters_filled['canonical']))

# Mapping canonical brand back to main dataset
df['brand'] = df['clean_name'].map(brand_map).fillna(df['clean_name'])

#I loaded my edited cluster file, split all fuzzy-matched aliases into individual rows,
#and mapped each one to its clean brand name using a dictionary. That’s how I created a consistent ‘brand’ column in the dataset


In [ ]:
def extract_variant(name):
    # Ensuring the input is a string before applying regex
    name_str = str(name)
    pattern = r'(\d+\.?\d*\s*(?:g|kg|l|ml))'
    match = re.search(pattern, name_str)
    return match.group(1).replace(" ", "") if match else 'unspecified'

df['variant'] = df['clean_name'].apply(extract_variant)

#I used regex to extract size or volume like 70g or 1L from each product name.
#This new 'variant' column helps me later calculate price per gram and classify products.”

In [ ]:
def convert_variant_to_grams(variant):
    if isinstance(variant, str):
        match = re.search(r'([\d\.]+)', variant)
        if match:
            value = float(match.group(1))
            if 'l' in variant.lower():
                return value * 1000  # 1L = 1000g
            return value  # Assume grams
    return np.nan

df['variant_g'] = df['variant'].apply(convert_variant_to_grams)


In [ ]:
df['price_per_gram'] = df['MRP'] / df['variant_g']


In [ ]:
df.head(20)

,Product ID,Product Name,Unit Sold,pcs/box,MRP,Selling Price,Region,Category Code,original_name,clean_name,brand,variant,variant_g,price_per_gram
0,76038834.0,Maggie Noodles 70g Masala,8.0,24.0,41.67,209.92,Kolkata,FMCG-NOO,Maggie Noodles 70g Masala,maggie noodles 70g masala,amul noodles 70g masala,70g,70.0,0.595286
1,97020468.0,Maggi Juice 1L Vanilla,60.0,48.0,62.23,3665.40,Hyderabad,FMCG-JUI,Maggi Juice 1L Vanilla,maggi juice 1l vanilla,amul juice 1l vanilla,1l,1000.0,0.062230
2,94530087.0,Britannia Milk 100g Mango,44.0,6.0,98.41,2744.28,Hyderabad,FMCG-MIL,Britannia Milk 100g Mango,britannia milk 100g mango,amul milk 100g mango,100g,100.0,0.984100
3,65974587.0,Nestle Noodles 1L Mango,100.0,24.0,40.45,3394.00,Hyderabad,FMCG-NOO,Nestle Noodles 1L Mango,nestle noodles 1l mango,amul noodles 1l mango,1l,1000.0,0.040450
4,92746917.0,Maggie Juice 500g Chocolate,11.0,24.0,62.45,451.00,Chennai,NaN,Maggie Juice 500g Chocolate,maggie juice 500g chocolate,amul juice 500g chocolate,500g,500.0,0.124900
5,46758737.0,Maggi Butter 200g,86.0,48.0,73.25,6166.20,Hyderabad,FMCG-BUT,Maggi Butter 200g,maggi butter 200g,maggi butter 100g,200g,200.0,0.366250
6,20951456.0,Britannia Noodles 200g Mango,29.0,6.0,100.00,1919.80,Chennai,NaN,Britannia Noodles 200g Mango,britannia noodles 200g mango,amul noodles 200g mango,200g,200.0,0.500000
7,56377233.0,Nestle Juice 70g Chocolate,49.0,6.0,23.88,786.94,Hyderabad,FMCG-JUI,Nestle Juice 70g Chocolate,nestle juice 70g chocolate,amul juice 70g chocolate,70g,70.0,0.341143
8,7672672.0,Britannia Butter 500g Vanilla,97.0,24.0,77.37,5851.04,Bangalore,FMCG-BUT,Britannia Butter 500g Vanilla,britannia butter 500g vanilla,amul butter 500g vanilla,500g,500.0,0.154740
9,26785542.0,Maggi Butter 500g Mango,100.0,12.0,44.37,3598.00,Mumbai,FMCG-BUT,Maggi Butter 500g Mango,maggi butter 500g mango,amul butter 500g mango,500g,500.0,0.088740


In [ ]:
df['MRP_imputed'] = df.groupby(['brand', 'variant', 'Region'])['MRP'].transform('mean')

# fallback: brand + variant
mask = df['MRP_imputed'].isna()
df.loc[mask, 'MRP_imputed'] = df[mask].groupby(['brand', 'variant'])['MRP'].transform('mean')

# fallback: brand only
mask = df['MRP_imputed'].isna()
df.loc[mask, 'MRP_imputed'] = df[mask].groupby('brand')['MRP'].transform('mean')



In [ ]:
df.head(40)

,Product ID,Product Name,Unit Sold,pcs/box,MRP,Selling Price,Region,Category Code,original_name,clean_name,brand,variant,variant_g,price_per_gram,MRP_imputed
0,76038834.0,Maggie Noodles 70g Masala,8.0,24.0,41.67,209.92,Kolkata,FMCG-NOO,Maggie Noodles 70g Masala,maggie noodles 70g masala,amul noodles 70g masala,70g,70.0,0.595286,45.316000
1,97020468.0,Maggi Juice 1L Vanilla,60.0,48.0,62.23,3665.40,Hyderabad,FMCG-JUI,Maggi Juice 1L Vanilla,maggi juice 1l vanilla,amul juice 1l vanilla,1l,1000.0,0.062230,39.416667
2,94530087.0,Britannia Milk 100g Mango,44.0,6.0,98.41,2744.28,Hyderabad,FMCG-MIL,Britannia Milk 100g Mango,britannia milk 100g mango,amul milk 100g mango,100g,100.0,0.984100,52.580000
3,65974587.0,Nestle Noodles 1L Mango,100.0,24.0,40.45,3394.00,Hyderabad,FMCG-NOO,Nestle Noodles 1L Mango,nestle noodles 1l mango,amul noodles 1l mango,1l,1000.0,0.040450,48.130000
4,92746917.0,Maggie Juice 500g Chocolate,11.0,24.0,62.45,451.00,Chennai,NaN,Maggie Juice 500g Chocolate,maggie juice 500g chocolate,amul juice 500g chocolate,500g,500.0,0.124900,41.265000
5,46758737.0,Maggi Butter 200g,86.0,48.0,73.25,6166.20,Hyderabad,FMCG-BUT,Maggi Butter 200g,maggi butter 200g,maggi butter 100g,200g,200.0,0.366250,73.250000
6,20951456.0,Britannia Noodles 200g Mango,29.0,6.0,100.00,1919.80,Chennai,NaN,Britannia Noodles 200g Mango,britannia noodles 200g mango,amul noodles 200g mango,200g,200.0,0.500000,76.776667
7,56377233.0,Nestle Juice 70g Chocolate,49.0,6.0,23.88,786.94,Hyderabad,FMCG-JUI,Nestle Juice 70g Chocolate,nestle juice 70g chocolate,amul juice 70g chocolate,70g,70.0,0.341143,23.880000
8,7672672.0,Britannia Butter 500g Vanilla,97.0,24.0,77.37,5851.04,Bangalore,FMCG-BUT,Britannia Butter 500g Vanilla,britannia butter 500g vanilla,amul butter 500g vanilla,500g,500.0,0.154740,77.370000
9,26785542.0,Maggi Butter 500g Mango,100.0,12.0,44.37,3598.00,Mumbai,FMCG-BUT,Maggi Butter 500g Mango,maggi butter 500g mango,amul butter 500g mango,500g,500.0,0.088740,44.370000


In [ ]:
from sklearn.linear_model import LinearRegression

# Separating data into training (non-missing current MRP) and testing (missing current MRP)
train_lr = df[df['MRP'].notna()].copy()
test_lr = df[df['MRP'].isna()].copy()

# Add MRP_imputed as a feature, handling potential NaNs in MRP_imputed before creating dummies
train_lr['MRP_imputed_feature'] = train_lr['MRP_imputed'].fillna(train_lr['MRP_imputed'].mean())
test_lr['MRP_imputed_feature'] = test_lr['MRP_imputed'].fillna(train_lr['MRP_imputed'].mean())


# Create dummy variables for brand, variant, region for both train and test sets
X_train_lr = pd.get_dummies(train_lr[['brand', 'variant', 'Region', 'MRP_imputed_feature']], dummy_na=False)
X_test_lr = pd.get_dummies(test_lr[['brand', 'variant', 'Region', 'MRP_imputed_feature']], dummy_na=False)


# Align columns - this is crucial to ensure both dataframes have the same columns
common_cols = list(set(X_train_lr.columns) & set(X_test_lr.columns))
X_train_lr = X_train_lr[common_cols]
X_test_lr = X_test_lr[common_cols].reindex(columns=X_train_lr.columns, fill_value=0)


# Fit regression model to predict original MRP
lr = LinearRegression()

# Checking if training data is not empty before fitting
if not X_train_lr.empty and not train_lr['MRP'].empty:
    lr.fit(X_train_lr, train_lr['MRP'])

# Predictimg missing MRP values using the regression model and impute the original 'MRP' column
# Check if testing data is not empty before predicting
    if not X_test_lr.empty:
        predicted_mrp = lr.predict(X_test_lr)
        df.loc[df['MRP'].isna(), 'MRP'] = predicted_mrp
    else:
        print("Warning: Testing data for linear regression is empty. No predictions made.")
else:
    print("Warning: Training data for linear regression is empty. Skipping this imputation step.")


#I used Linear Regression to fill the remaining missing MRPs by training on brand, variant, region, and imputed MRP.
#This gave me more accurate and interpretable estimates than just using averages.”

In [ ]:
print("Missing MRP values:", df['MRP'].isna().sum())


Missing MRP values: 0


In [ ]:
# Fill any remaining NaN values in 'MRP' using the mean imputation fallback
# final fallback after linear regression imputation
df['MRP'] = df['MRP'].fillna(df['MRP_imputed'])

In [ ]:
df.head(20)

,Product ID,Product Name,Unit Sold,pcs/box,MRP,Selling Price,Region,Category Code,original_name,clean_name,brand,variant,variant_g,price_per_gram,MRP_imputed
0,76038834.0,Maggie Noodles 70g Masala,8.0,24.0,41.670000,209.92,Kolkata,FMCG-NOO,Maggie Noodles 70g Masala,maggie noodles 70g masala,amul noodles 70g masala,70g,70.0,0.595286,45.316000
1,97020468.0,Maggi Juice 1L Vanilla,60.0,48.0,62.230000,3665.40,Hyderabad,FMCG-JUI,Maggi Juice 1L Vanilla,maggi juice 1l vanilla,amul juice 1l vanilla,1l,1000.0,0.062230,39.416667
2,94530087.0,Britannia Milk 100g Mango,44.0,6.0,98.410000,2744.28,Hyderabad,FMCG-MIL,Britannia Milk 100g Mango,britannia milk 100g mango,amul milk 100g mango,100g,100.0,0.984100,52.580000
3,65974587.0,Nestle Noodles 1L Mango,100.0,24.0,40.450000,3394.00,Hyderabad,FMCG-NOO,Nestle Noodles 1L Mango,nestle noodles 1l mango,amul noodles 1l mango,1l,1000.0,0.040450,48.130000
4,92746917.0,Maggie Juice 500g Chocolate,11.0,24.0,62.450000,451.00,Chennai,NaN,Maggie Juice 500g Chocolate,maggie juice 500g chocolate,amul juice 500g chocolate,500g,500.0,0.124900,41.265000
5,46758737.0,Maggi Butter 200g,86.0,48.0,73.250000,6166.20,Hyderabad,FMCG-BUT,Maggi Butter 200g,maggi butter 200g,maggi butter 100g,200g,200.0,0.366250,73.250000
6,20951456.0,Britannia Noodles 200g Mango,29.0,6.0,100.000000,1919.80,Chennai,NaN,Britannia Noodles 200g Mango,britannia noodles 200g mango,amul noodles 200g mango,200g,200.0,0.500000,76.776667
7,56377233.0,Nestle Juice 70g Chocolate,49.0,6.0,23.880000,786.94,Hyderabad,FMCG-JUI,Nestle Juice 70g Chocolate,nestle juice 70g chocolate,amul juice 70g chocolate,70g,70.0,0.341143,23.880000
8,7672672.0,Britannia Butter 500g Vanilla,97.0,24.0,77.370000,5851.04,Bangalore,FMCG-BUT,Britannia Butter 500g Vanilla,britannia butter 500g vanilla,amul butter 500g vanilla,500g,500.0,0.154740,77.370000
9,26785542.0,Maggi Butter 500g Mango,100.0,12.0,44.370000,3598.00,Mumbai,FMCG-BUT,Maggi Butter 500g Mango,maggi butter 500g mango,amul butter 500g mango,500g,500.0,0.088740,44.370000


In [ ]:
print("Remaining missing values in MRP:", df['MRP'].isna().sum())


Remaining missing values in MRP: 0


In [ ]:
df['expected_SP'] = df['Unit Sold'] * df['MRP_imputed']
df['SP_deviation_%'] = ((df['Selling Price'] - df['expected_SP']) / df['expected_SP']) * 100

# to flag large deviation cases
df['SP_outlier_flag'] = df['SP_deviation_%'].abs() > 20


In [ ]:
df[['Product ID', 'Product Name', 'brand', 'variant', 'Unit Sold', 'MRP', 'MRP_imputed',
    'Selling Price', 'expected_SP', 'SP_deviation_%', 'SP_outlier_flag']].head(20)


,Product ID,Product Name,brand,variant,Unit Sold,MRP,MRP_imputed,Selling Price,expected_SP,SP_deviation_%,SP_outlier_flag
0,76038834.0,Maggie Noodles 70g Masala,amul noodles 70g masala,70g,8.0,41.670000,45.316000,209.92,362.528000,-42.095507,True
1,97020468.0,Maggi Juice 1L Vanilla,amul juice 1l vanilla,1l,60.0,62.230000,39.416667,3665.40,2365.000000,54.985201,True
2,94530087.0,Britannia Milk 100g Mango,amul milk 100g mango,100g,44.0,98.410000,52.580000,2744.28,2313.520000,18.619247,False
3,65974587.0,Nestle Noodles 1L Mango,amul noodles 1l mango,1l,100.0,40.450000,48.130000,3394.00,4813.000000,-29.482651,True
4,92746917.0,Maggie Juice 500g Chocolate,amul juice 500g chocolate,500g,11.0,62.450000,41.265000,451.00,453.915000,-0.642191,False
5,46758737.0,Maggi Butter 200g,maggi butter 100g,200g,86.0,73.250000,73.250000,6166.20,6299.500000,-2.116041,False
6,20951456.0,Britannia Noodles 200g Mango,amul noodles 200g mango,200g,29.0,100.000000,76.776667,1919.80,2226.523333,-13.775887,False
7,56377233.0,Nestle Juice 70g Chocolate,amul juice 70g chocolate,70g,49.0,23.880000,23.880000,786.94,1170.120000,-32.747069,True
8,7672672.0,Britannia Butter 500g Vanilla,amul butter 500g vanilla,500g,97.0,77.370000,77.370000,5851.04,7504.890000,-22.036965,True
9,26785542.0,Maggi Butter 500g Mango,amul butter 500g mango,500g,100.0,44.370000,44.370000,3598.00,4437.000000,-18.909173,False


In [ ]:
median_dev = df.dropna(subset=['SP_deviation_%']).groupby(['brand', 'variant'])['SP_deviation_%'].median()
#I calculated the median pricing deviation for each brand and variant to understand how each product usually behaves — and used it later to impute missing Selling Prices

In [ ]:
# Function to fill SP based on brand/variant deviation
def impute_sp(row):
    if pd.notna(row['Selling Price']):
        return row['Selling Price']  # keep original
    dev = median_dev.get((row['brand'], row['variant']), 0)  # fallback = 0% deviation
    return row['expected_SP'] * (1 + dev / 100)

df['Selling Price'] = df.apply(impute_sp, axis=1)
#“I filled missing Selling Prices by adjusting the expected price using that product’s typical pricing deviation
#so it reflects how it usually performs in the market.

In [ ]:
print("Remaining missing Selling Price:", df['Selling Price'].isna().sum())


Remaining missing Selling Price: 3


In [ ]:
df.loc[df['Selling Price'].isna(), 'Selling Price'] = df.loc[df['Selling Price'].isna(), 'expected_SP']


In [ ]:
print("Remaining missing Selling Price:", df['Selling Price'].isna().sum())


Remaining missing Selling Price: 3


In [ ]:
df = df[df['Selling Price'].notna()].reset_index(drop=True)
print("✅ Rows after dropping missing Selling Price:", len(df))


✅ Rows after dropping missing Selling Price: 3999


In [ ]:
# Get dominant region by brand + variant if ≥90% rows have same region
region_counts = (
    df[df['Region'].notna()]
    .groupby(['brand', 'variant', 'Region'])
    .size()
    .reset_index(name='count')
)

# Total per brand+variant
total_counts = region_counts.groupby(['brand', 'variant'])['count'].sum().reset_index(name='total')

# Merge to calculate dominance
region_dominance = region_counts.merge(total_counts, on=['brand', 'variant'])
region_dominance['ratio'] = region_dominance['count'] / region_dominance['total']

# Keep only brand+variant pairs with ≥90% region dominance
dominant_regions = region_dominance[region_dominance['ratio'] >= 0.9]
dominant_region_map = dict(
    zip(zip(dominant_regions['brand'], dominant_regions['variant']), dominant_regions['Region'])
)


#I identified dominant regions for each product — if 90% or more rows came from one region,
#I used that as the default region to fill missing ones


In [ ]:
def impute_region(row):
    if pd.notna(row['Region']):
        return row['Region']
    return dominant_region_map.get((row['brand'], row['variant']), 'Unknown')

df['Region'] = df.apply(impute_region, axis=1)

#I filled missing Regions using the most common region for that product
#but only if it covered 90% or more cases. Otherwise, I marked it as 'Unknown'.


In [ ]:
print("✅ Remaining missing Region:", df['Region'].isna().sum())
print("🔍 Region = 'Unknown':", (df['Region'] == 'Unknown').sum())


✅ Remaining missing Region: 0
🔍 Region = 'Unknown': 208


In [ ]:
# Fix 'nan' string if any
df['Category Code'] = df['Category Code'].replace('nan', np.nan)

# Get most common category per brand + variant
category_mode = (
    df[df['Category Code'].notna()]
    .groupby(['brand', 'variant'])['Category Code']
    .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else 'Unknown')
)

def impute_category(row):
    if pd.notna(row['Category Code']):
        return row['Category Code']
    return category_mode.get((row['brand'], row['variant']), 'Unknown')

df['Category Code'] = df.apply(impute_category, axis=1)

# mark imputed ones
df['category_imputed'] = df['Category Code'].isna()


#I filled missing Category Codes using the most common one for that product.
#If not found, I marked it as 'Unknown' and tracked which rows were filled.

In [ ]:
print("✅ Remaining missing Category Code:", df['Category Code'].isna().sum())


✅ Remaining missing Category Code: 0


In [ ]:
#To classify products into tiers fairly, I used price per gram as a normalization metric.
# But I didn’t apply the thresholds globally — because a gram of butter isn’t priced like a gram of juice.
# So instead, I grouped products by category and then applied quantile thresholds (40th and 80th percentile) within each group.
# That way, I could classify products as Entry-Level, Standard, or Premium relative to their direct competitors.”

# Function to assign tier within category group
def assign_tier(sub_df):
    q40 = sub_df['price_per_gram'].quantile(0.4)
    q80 = sub_df['price_per_gram'].quantile(0.8)

#I used quantile thresholds to divide each product category into pricing tiers.
# Quantiles help me segment products relative to how expensive they are within their own category — without using any arbitrary pricing cutoffs.

    def tier_label(p):
        if p <= q40:
            return 'Entry-Level'
        elif p <= q80:
            return 'Standard'
        else:
            return 'Premium'

    return sub_df['price_per_gram'].apply(tier_label)

# Apply this within each Category Code group
df['Product Tier'] = df.groupby('Category Code', group_keys=False).apply(assign_tier)


/tmp/ipython-input-32-518781978.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['Product Tier'] = df.groupby('Category Code', group_keys=False).apply(assign_tier)


In [ ]:
def assign_tier(sub_df):
    q40 = sub_df['price_per_gram'].quantile(0.4)
    q80 = sub_df['price_per_gram'].quantile(0.8)

    def tier_label(p):
        if p <= q40:
            return 'Entry-Level'
        elif p <= q80:
            return 'Standard'
        else:
            return 'Premium'

    return sub_df['price_per_gram'].apply(tier_label)

df['Product Tier'] = df.groupby('Category Code', group_keys=False).apply(assign_tier)


/tmp/ipython-input-33-2490274437.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['Product Tier'] = df.groupby('Category Code', group_keys=False).apply(assign_tier)


In [ ]:
# Check number of products in each Product Tier
print(df['Product Tier'].value_counts())

# Check how Product Tier is distributed across different categories
product_tier_summary = pd.crosstab(df['Category Code'], df['Product Tier'])
print(product_tier_summary)


Product Tier
Entry-Level    1518
Standard       1513
Premium         968
Name: count, dtype: int64
Product Tier   Entry-Level  Premium  Standard
Category Code                                
FMCG-BIS               307      195       306
FMCG-BUT               311      198       310
FMCG-JUI               299      191       298
FMCG-MIL               298      199       298
FMCG-NOO               300      184       299
Unknown                  3        1         2


In [ ]:
# Total revenue by SKU
sku_revenue = df.groupby('Product Name')['Selling Price'].sum().sort_values(ascending=False)

# Normalize to quantiles for classification
revenue_q40 = sku_revenue.quantile(0.4)
revenue_q80 = sku_revenue.quantile(0.8)

# Define a mapping function
def revenue_tier(rev):
    if rev <= revenue_q40:
        return 'Low Revenue'
    elif rev <= revenue_q80:
        return 'Medium Revenue'
    else:
        return 'High Revenue'

# Create a mapping dictionary
revenue_tier_map = sku_revenue.apply(revenue_tier).to_dict()

# Add to your df
df['Revenue Tier'] = df['Product Name'].map(revenue_tier_map)

#Classifies each product as Low/Medium/High Revenue based on total sales.
#Uses 40th and 80th percentile cutoffs for balance.


In [ ]:
tier_check = pd.crosstab(df['Product Tier'], df['Revenue Tier'])
print(tier_check)


Revenue Tier  High Revenue  Low Revenue  Medium Revenue
Product Tier                                           
Entry-Level            428          417             673
Premium                338          182             448
Standard               519          331             663


In [ ]:
volume_per_sku = df.groupby('Product Name')['Unit Sold'].sum().sort_values(ascending=False)

# Define quantiles
vol_q40 = volume_per_sku.quantile(0.4)
vol_q80 = volume_per_sku.quantile(0.8)

# Define function to classify by volume
def classify_volume(units):
    if units <= vol_q40:
        return 'Low Volume'
    elif units <= vol_q80:
        return 'Medium Volume'
    else:
        return 'High Volume'

volume_tier_map = volume_per_sku.apply(classify_volume).to_dict()

# Add new tier column to df
df['Volume Tier'] = df['Product Name'].map(volume_tier_map)

#Classifies each product as Low/Medium/High Volume based on total quantity sold.
#Complements revenue tier to analyze performance from a unit-sales perspective.

In [ ]:
def final_tier(row):
    score = 0
    if row['Product Tier'] == 'Premium':
        score += 2
    elif row['Product Tier'] == 'Standard':
        score += 1

    if row['Revenue Tier'] == 'High Revenue':
        score += 2
    elif row['Revenue Tier'] == 'Medium Revenue':
        score += 1

    if row['Volume Tier'] == 'High Volume':
        score += 2
    elif row['Volume Tier'] == 'Medium Volume':
        score += 1

    if score >= 5:
        return 'Premium'
    elif score >= 3:
        return 'Standard'
    else:
        return 'Entry-Level'

df['Final Product Tier'] = df.apply(final_tier, axis=1)

#Instead of relying on just one metric, I created a composite scoring system that considers product pricing, revenue performance, and volume sold.
#This gave me a more balanced and business-relevant Final Product Tier.


In [ ]:
df[['Product Name', 'Product Tier', 'Revenue Tier', 'Volume Tier', 'Final Product Tier']].head(10)


,Product Name,Product Tier,Revenue Tier,Volume Tier,Final Product Tier
0,Maggie Noodles 70g Masala,Standard,High Revenue,High Volume,Premium
1,Maggi Juice 1L Vanilla,Entry-Level,High Revenue,High Volume,Standard
2,Britannia Milk 100g Mango,Premium,High Revenue,High Volume,Premium
3,Nestle Noodles 1L Mango,Entry-Level,High Revenue,High Volume,Standard
4,Maggie Juice 500g Chocolate,Entry-Level,Low Revenue,Medium Volume,Entry-Level
5,Maggi Butter 200g,Standard,Medium Revenue,Medium Volume,Standard
6,Britannia Noodles 200g Mango,Standard,High Revenue,High Volume,Premium
7,Nestle Juice 70g Chocolate,Standard,Medium Revenue,Medium Volume,Standard
8,Britannia Butter 500g Vanilla,Standard,Medium Revenue,Medium Volume,Standard
9,Maggi Butter 500g Mango,Entry-Level,Low Revenue,Low Volume,Entry-Level


Region wise sales summary

In [ ]:
# Total revenue and total units sold by region
region_summary = df.groupby('Region').agg({
    'Selling Price': 'sum',
    'Unit Sold': 'sum'
}).sort_values(by='Selling Price', ascending=False)

region_summary.columns = ['Total Revenue', 'Total Units Sold']
display(region_summary)


,Total Revenue,Total Units Sold
Region,,
Kolkata,1.482938e+06,34066.0
Hyderabad,1.474398e+06,33556.0
Mumbai,1.440242e+06,31456.0
Delhi,1.404774e+06,32290.0
Bangalore,1.370868e+06,30567.0
Chennai,1.357889e+06,31404.0
Unknown,4.197112e+05,10025.0


Product-wise Sales Summary

In [ ]:
# Total revenue, units sold, and average MRP by product
product_summary = df.groupby('Product Name').agg({
    'Selling Price': 'sum',
    'Unit Sold': 'sum',
    'MRP': 'mean'
}).sort_values(by='Selling Price', ascending=False)

product_summary.columns = ['Total Revenue', 'Total Units Sold', 'Avg MRP']
display(product_summary.head(10))  # Top 10 products


,Total Revenue,Total Units Sold,Avg MRP
Product Name,,,
Maggie Biscuits 500g Chocolate,39341.290000,766.0,59.635385
Parle Juice 200g Vanilla,32303.570000,643.0,70.740000
Britannia Biscuits 70g Chocolate,32056.850000,747.0,51.331385
Nestle Milk 500g Mango,31371.510000,558.0,55.763000
Maggie Juice 200g Masala,30241.990000,612.0,56.587625
Maggie Noodles 500g Mango,29875.920000,511.0,69.522857
Britannia Milk 100g Mango,28745.166131,546.0,60.830000
Britannia Noodles 100g Plain,28250.710000,453.0,78.980000
Maggi Milk 70g Plain,28160.810000,503.0,66.758889


In [ ]:
# Top 5 revenue-generating products
top5_products = product_summary.head(5)
print("Top 5 Products by Revenue:")
display(top5_products)


Top 5 Products by Revenue:


,Total Revenue,Total Units Sold,Avg MRP
Product Name,,,
Maggie Biscuits 500g Chocolate,39341.29,766.0,59.635385
Parle Juice 200g Vanilla,32303.57,643.0,70.740000
Britannia Biscuits 70g Chocolate,32056.85,747.0,51.331385
Nestle Milk 500g Mango,31371.51,558.0,55.763000
Maggie Juice 200g Masala,30241.99,612.0,56.587625


In [ ]:
# Estimate expected revenue and deviation
df['Expected Revenue'] = df['Unit Sold'] * df['MRP']
df['Revenue Deviation (%)'] = 100 * (df['Expected Revenue'] - df['Selling Price']) / df['Expected Revenue']

# Flag rows where actual selling price deviates more than 30% from expected
anomalies = df[abs(df['Revenue Deviation (%)']) > 30]
print(f"Found {len(anomalies)} pricing anomalies.")
display(anomalies[['Product Name', 'Region', 'MRP', 'Unit Sold', 'Selling Price', 'Expected Revenue', 'Revenue Deviation (%)']])


Found 1133 pricing anomalies.


,Product Name,Region,MRP,Unit Sold,Selling Price,Expected Revenue,Revenue Deviation (%)
0,Maggie Noodles 70g Masala,Kolkata,41.670,8.0,209.92,333.360,37.029038
2,Britannia Milk 100g Mango,Hyderabad,98.410,44.0,2744.28,4330.040,36.622294
4,Maggie Juice 500g Chocolate,Chennai,62.450,11.0,451.00,686.950,34.347478
6,Britannia Noodles 200g Mango,Chennai,100.000,29.0,1919.80,2900.000,33.800000
7,Nestle Juice 70g Chocolate,Hyderabad,23.880,49.0,786.94,1170.120,32.747069
...,...,...,...,...,...,...,...
3957,Nestle Butter 100g Chocolate,Mumbai,37.125,19.0,1253.62,705.375,-77.723906
3958,Amul Juice 100g Vanilla,Mumbai,17.200,88.0,991.76,1513.600,34.476744
3967,Parle Juice 200g Mango,Mumbai,70.640,15.0,691.05,1059.600,34.781993
3973,Maggie Milk 100g Plain,Delhi,88.480,69.0,4099.98,6105.120,32.843580


In [ ]:
# Median values by category and final product tier
optimal_packaging = df.groupby(['Category Code', 'Final Product Tier']).agg({
    'pcs/box': 'median',
    'MRP': 'median',
    'Selling Price': 'median'
}).rename(columns={
    'pcs/box': 'Recommended Pack Size',
    'MRP': 'Recommended MRP',
    'Selling Price': 'Recommended SP'
}).reset_index()

display(optimal_packaging)


,Category Code,Final Product Tier,Recommended Pack Size,Recommended MRP,Recommended SP
0,FMCG-BIS,Entry-Level,12.0,46.470000,1233.600000
1,FMCG-BIS,Premium,12.0,68.030000,2454.560000
2,FMCG-BIS,Standard,12.0,57.845000,2091.870000
3,FMCG-BUT,Entry-Level,12.0,50.330000,1480.360000
4,FMCG-BUT,Premium,24.0,62.811000,2576.751483
5,FMCG-BUT,Standard,12.0,55.204946,1688.620000
6,FMCG-JUI,Entry-Level,18.0,48.995000,1309.740000
7,FMCG-JUI,Premium,12.0,67.395000,2692.490000
8,FMCG-JUI,Standard,24.0,57.805000,1850.270000
9,FMCG-MIL,Entry-Level,12.0,49.190000,1467.550000


In [ ]:
# Total revenue by region and tier
tier_region_summary = df.groupby(['Region', 'Final Product Tier'])['Selling Price'].sum().unstack().fillna(0)
display(tier_region_summary)


Final Product Tier,Entry-Level,Premium,Standard
Region,,,
Bangalore,381137.167040,323144.916307,666586.087809
Chennai,420318.092271,383006.495784,554564.091108
Delhi,442548.294296,375873.883086,586352.305221
Hyderabad,410004.590248,425547.117658,638846.038404
Kolkata,424224.524582,381228.706353,677485.172576
Mumbai,398737.762798,398987.368289,642517.344025
Unknown,119936.949385,115418.385107,184355.891421


In [ ]:
underperforming_premium = df[(df['Final Product Tier'] == 'Premium') & (df['Revenue Tier'] == 'Low Revenue')]
print(f"Found {underperforming_premium.shape[0]} underperforming Premium products.")
display(underperforming_premium[['Product Name', 'Region', 'Unit Sold', 'Selling Price']].drop_duplicates())



Found 0 underperforming Premium products.


,Product Name,Region,Unit Sold,Selling Price
